In [220]:
    """
 1. Написать на PyTorch глубокую сеть. Проверить работу форвард пасса.
 2. Написать адаптивный оптимизатор.
 3. Решить задачу нахождения корней квадратного уравнения методом градиентного спуска.
    """ 

'\n 1. Написать на PyTorch глубокую сеть. Проверить работу форвард пасса.\n 2. Написать адаптивный оптимизатор.\n 3. Решить задачу нахождения корней квадратного уравнения методом градиентного спуска.\n'

In [221]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

In [222]:
class MyModelNN(nn.Module):
    def __init__(self,layer_in,layer2,layer3,layer_out,):
        
        super(MyModelNN, self).__init__()
        
        self.fc1 = nn.Linear(layer_in, layer2)
        self.fc2 = nn.Linear(layer2, layer3)
        self.fc3 = nn.Linear(layer3, layer3)
        self.fc4 = nn.Linear(layer3, layer_out)
        
        self.sig_ = nn.Sigmoid()
        self.relu_ = nn.ReLU()
    
    def forward(self, x, y):
        x = self.relu_(self.fc1(x))
        x = self.relu_(self.fc2(x))
        x = self.relu_(self.fc3(x))
        x = self.relu_(x)
        return x
    
    def forward(self, x):
        x = self.sig_(self.fc1(x))
        x = self.sig_(self.fc2(x))
        x = self.sig_(self.fc3(x))
        x = self.fc4(x)
        return x


In [223]:
model = MyModelNN(4, 5, 7,4)
model

MyModelNN(
  (fc1): Linear(in_features=4, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=7, bias=True)
  (fc3): Linear(in_features=7, out_features=7, bias=True)
  (fc4): Linear(in_features=7, out_features=4, bias=True)
  (sig_): Sigmoid()
  (relu_): ReLU()
)

In [224]:
X = torch.randn((200, 4))
X[:4,:4]

tensor([[-0.2219, -0.5366, -2.0597,  1.0053],
        [ 0.2618, -1.4053,  2.1147, -2.3223],
        [ 0.3316, -0.3065,  1.1659,  1.5225],
        [ 1.0030, -0.5749,  0.2629, -1.1119]])

In [225]:
y = model(X)
y[:4,:4]

tensor([[ 0.1161, -0.3180,  0.0957,  0.0669],
        [ 0.1140, -0.3258,  0.0923,  0.0642],
        [ 0.1154, -0.3181,  0.0953,  0.0679],
        [ 0.1153, -0.3207,  0.0943,  0.0662]], grad_fn=<SliceBackward0>)

In [226]:
# 2. Написать адаптивный оптимизатор

In [227]:
"""
  Adam (adaptive moment estimation) - это алгоритм оптимизации, совмещающий принципы инерции MomentumSGD и 
адаптивного обновления параметров AdaGrad и его модификаций.
  Для реализации алгоритма необходимо использование подхода экспоненциально затухающего бегущего 
среднего для градиентов целевой функции и их квадратов.
  Чтобы избавиться от этой проблемы и не вводить новые гиперпараметры, оценки первого и второго моментов немного видоизменяются.
        """

class Adam:
        
    def __init__(self, params, alpha=1e-3, beta1=0.9, beta2=0.999, epsilon=1e-8, nodeinfo=None):
        self.model = model
        self.alpha = alpha
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.epsilon = nodeinfo
        self.params = params
        
        self.s = {}
        self.v = {}
        
        for i in range(len(params)//2 ):
            self.v["dW" + str(i)] = np.zeros(params["W" + str(i)].shape)
            self.v["db" + str(i)] = np.zeros(params["b" + str(i)].shape)

            self.s["dW" + str(i)] = np.zeros(params["W" + str(i)].shape)
            self.s["db" + str(i)] = np.zeros(params["b" + str(i)].shape)

    
    def update_params_with_Adam(self, grads, learning_rate,t):
        epsilon = pow(10,-8)
        v_corrected = {}                         
        s_corrected = {}
        params = self.params
        
        for l in range(len(params) // 2 ):
#           оценка первого момента (среднее градиентов)
            self.v["dW" + str(l)] = self.beta1 * self.v["dW" + str(l)] + (1 - self.beta1) * grads['dW' + str(l)]
            self.v["db" + str(l)] = self.beta1 * self.v["db" + str(l)] + (1 - self.beta1) * grads['db' + str(l)]

            v_corrected["dW" + str(l)] = self.v["dW" + str(l)] / (1 - np.power(self.beta1, t))
            v_corrected["db" + str(l)] = self.v["db" + str(l)] / (1 - np.power(self.beta1, t))

#           оценка второго момента (средняя нецентрированная дисперсия градиентов)
            self.s["dW" + str(l)] = self.beta2 * self.s["dW" + str(l)] + (1 - self.beta2) * np.power(grads['dW' + str(l)], 2)
            self.s["db" + str(l)] = self.beta2 * self.s["db" + str(l)] + (1 - self.beta2) * np.power(grads['db' + str(l)], 2)

#           коррекция смещения
            s_corrected["dW" + str(l)] = self.s["dW" + str(l)] / (1 - np.power(self.beta2, t))
            s_corrected["db" + str(l)] = self.s["db" + str(l)] / (1 - np.power(self.beta2, t))
        
#           обновление весов и смещение
            params["W" + str(l)] = params["W" + str(l)] - learning_rate * v_corrected["dW" + str(l)] / np.sqrt(s_corrected["dW" + str(l)] + epsilon)
            params["b" + str(l)] = params["b" + str(l)] - learning_rate * v_corrected["db" + str(l)] / np.sqrt(s_corrected["db" + str(l)] + epsilon)
        return params

# Его преимущества:
#     Простая реализация.
#     Вычислительная эффективность.
#     Небольшие требования к памяти.
#     Инвариант к диагональному масштабированию градиентов.
#     Хорошо подходит для больших с точки зрения данных и параметров задач.
#     Подходит для нестационарных целей.
#     Подходит для задач с очень шумными или разреженными градиентами.
#     Гиперпараметры имеют наглядную интерпретацию и обычно требуют небольшой настройки.



In [228]:
# 3. Решить задачу нахождения корней квадратного уравнения методом градиентного спуска
# x ** 2 - 6 * x + 4 = 0

In [229]:
from sympy import *
import numpy as np

a = 1
b = -6
c = 4

x = Symbol('x')
y = (x ** 2 - 6 * x + 4)**2

yprime = y.diff(x)
print(yprime)

(4*x - 12)*(x**2 - 6*x + 4)


In [230]:
# заданная функция
def f1(x):
    return x**2 - 6*x + 4    

# производная функции
def grad1(x):
    return 2 * x - 6

# метод градиентного спуска 
def gradient_descent(x, lr, acc):
    x1 = x
    x2 = x1 - lr * grad1(x1)
    count = 1
    while f1(x2) > acc:
        x1 = x2
        x2 = x1 - lr * grad1(x1)
        count += 1
    return round(x2,4)
    
print(gradient_descent(-10, 0.00001, 0.00001))
gradient_descent(10, 0.00001, 0.00001)   

0.7639


5.2361

In [231]:
z,x = 0,0

class SDGMomentum:
    def __init__(self, momentum, lr, model):
        self.momentum = momentum
        self.lr = lr
        self.velocity = torch.zeros_like(model)
        self.model = model

    def step(self, grad):
        self.velocity = self.momentum * self.velocity - self.lr * grad
        self.model += self.velocity

x = torch.tensor(10.)
z = solver(init_x=x, optimizer=SDGMomentum(lr=0.00001, momentum=0.96, model=x))

x = torch.tensor(-10.)
z = solver(init_x=x, optimizer=SDGMomentum(lr=0.001, momentum=0.96, model=x))

tensor(5.2361)
tensor(0.7639)


In [232]:
z,x = 0,0

class AdaGrad:
    def __init__(self, lr, model):
        self.accumulated = torch.zeros_like(model)
        self.lr = lr
        self.adapt_lr = lr
        self.model = model

    def step(self, grad):
        self.accumulated += grad**2
        self.adapt_lr = self.lr / torch.sqrt(self.accumulated)
        self.model -= self.adapt_lr * grad

x = torch.tensor(10.)
z = solver(init_x=x, optimizer=AdaGrad(lr=0.7, model=x), max_iter=10000)

x = torch.tensor(-10.)
z = solver(init_x=x, optimizer=AdaGrad(lr=0.7, model=x), max_iter=10000)

tensor(5.2361)
tensor(0.7639)


In [233]:
z,x = 0,0

class RMSprop:
    def __init__(self, rho, lr, model):
        self.accumulated = torch.zeros_like(model)
        self.rho = rho
        self.lr = lr
        self.adapt_lr = lr
        self.model = model

    def step(self, grad):
        self.accumulated += self.rho * self.accumulated + (1 - self.rho) * grad**2
        self.adapt_lr = self.lr / torch.sqrt(self.accumulated)
        self.model -= self.adapt_lr * grad
        
x = torch.tensor(-10.)
z = solver(init_x=x, optimizer=RMSprop(lr=0.9, rho=0.0001, model=x), max_iter=10000)

x = torch.tensor(10.)
z = solver(init_x=x, optimizer=RMSprop(lr=0.9, rho=0.0001, model=x), max_iter=10000)

tensor(0.7639)
tensor(5.2361)


In [234]:
# 1 всегда ли сойдемся за приемлемое количество шагов?
# 2 важна ли начальная точка?
# 3 как найти второй корень?
# 4 как вляет ЛР?

a = """
1 Нет. Для каждого оптимизатора необходимо подбирать кол-во шагов и лернинг рейт, чтобы ответ был схожим.
2 Важна. В уравнении имеется 2 корня. От начальной точки и кол-ва шагов будет прослеживаться разный результат. 
3 По теореме Виета. Подставляя различные в первоначальную точку можно обнаружить разные корни уравнения.
4 На скорость приближения к минимуму.
"""
print(a)


1 Нет. Для каждого оптимизатора необходимо подбирать кол-во шагов и лернинг рейт, чтобы ответ был схожим.
2 Важна. В уравнении имеется 2 корня. От начальной точки и кол-ва шагов будет прослеживаться разный результат. 
3 По теореме Виета. Подставляя различные в первоначальную точку можно обнаружить разные корни уравнения.
4 На скорость приближения к минимуму.

